In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as lines
from stride import *
from stride.utils import wavelets
from utils import analytical_2d
import mosaic
from mosaic import *
import asyncio
import logging
%matplotlib notebook
#from mosaic import *

In [2]:
import mosaic

from stride import *
from stride.utils import wavelets

%matplotlib notebook

In [3]:
await mosaic.interactive('on',num_workers=5,platform="nvidia-acc")
runtime = mosaic.runtime()

HEAD            Listening at <CommsManager object at 140581589269200, uid=head, address=10.107.42.21, port=3001, state=listening>
MONITOR         Listening at <CommsManager object at 140579540525968, uid=monitor, address=10.107.42.21, port=3002, state=listening>
NODE:0          Listening at <CommsManager object at 140579540239184, uid=node:0, address=10.107.42.21, port=3003, state=listening>


In order to run the corresponding inverse problem, we also need to define a similar spatio-temporal grid and a `Problem`.

Then, our first step is to define this grid by creating a `Space` and `Time` objects that define the discretisation of our grid.

In [4]:
# Create the grid
#shape = (356, 385)
shape = (256, 256)
extra = (50, 50)
#absorbing = (40, 40)
absorbing = (0, 0)
spacing = (0.5e-3, 0.5e-3)#(0.015e-3, 0.015e-3)#(0.09e-3,0.09e-3)#(0.5e-3,0.5e-3)#(0.01e-3,0.01e-3)#(0.09e-3,0.09e-3)#(0.45e-3, 0.45e-3)

space = Space(shape=shape,
              extra=extra,
              absorbing=absorbing,
              spacing=spacing)

start = 0.
step =  0.1e-6#0.1e-6#0.0009e-6#0.08e-6#0.0009e-6#0.0065e-6#0.0350e-6
num = 10000

time = Time(start=start,
            step=step,
            num=num)

Based on this grid, we can define a `Problem`. In stride, a `Problem` object encapsulates all the information that is needed to describe a tomography experiment.

In its most basic form, the problem is given a name and the grid we have just created:

In [5]:
# Create problem
problem = Problem(name='anastasio2D-prostate',
                  space=space, time=time)

In [6]:
print(problem)

We can now start defining each of the components of the `Problem`.

Firstly, we need to define the physical properties that describe the medium in which our experiment is taking place.

Because we are going to run the default isotropic acoustic wave equation with constant density, we only need to define the longitudinal speed of sound of our medium. The longitudinal speed of sound is a scalar field, for which we can use the class `ScalarField`:

In [7]:
# # Create medium
# # Field(..., grid=problem.grid) is equivalent to Field(..., space=space, time=time)
# vp = ScalarField(name='vp', grid=problem.grid)
# vp.fill(1500.)

# problem.medium.add(vp)

Unlike our forward example, the speed-of-sound field is unknown to us at this point. So, we define our starting guess for the speed of sound to be homogeneous, with a value of 1500 m/s.

Next, we need to define the types of ultrasound transducers that we will use.

Because we are happy with using omnidirectional point sources and receivers, we can configure the default transducers:

In [8]:
# v_init = np.ones((150,150))
# v_init = 3500*v_init[0:75,:]
# v_init = 2500*v_init[75:,:]
# np.save('./v_init.npy', v_init) 

In [9]:
# Create medium
# Field(..., grid=problem.grid) is equivalent to Field(..., space=space, time=time)
vp = ScalarField.parameter(name='vp', grid=problem.grid, needs_grad=True)
#vp.fill(3500.)
vp.load('Med_blur.h5')
problem.medium.add(vp)

In [10]:
# rho = ScalarField(name='rho', grid=problem.grid)#,needs_grad=True)
# rho.fill(1000.)
# problem.medium.add(rho)

In [11]:
# alpha = ScalarField(name='alpha', grid=problem.grid)
# #alpha.fill(1.)
# alpha.load('./Med_Att_2p5.h5')
# problem.medium.add(alpha)

WORKER:0:0      Listening at <CommsManager object at 140579540169936, uid=worker:0:0, address=10.107.42.21, port=3004, state=listening>
WORKER:0:1      Listening at <CommsManager object at 140579540171536, uid=worker:0:1, address=10.107.42.21, port=3005, state=listening>
WORKER:0:2      Listening at <CommsManager object at 140579540172624, uid=worker:0:2, address=10.107.42.21, port=3006, state=listening>
WORKER:0:3      Listening at <CommsManager object at 140579540288080, uid=worker:0:3, address=10.107.42.21, port=3007, state=listening>
WORKER:0:4      Listening at <CommsManager object at 140579540289488, uid=worker:0:4, address=10.107.42.21, port=3008, state=listening>


In [12]:
# Create transducers
problem.transducers.default()

In order to use the transducers in our ultrasound acquisition, we need to add them to a `Geometry`. The `Geometry` defines the spatial location of each transducer within our grid.

In this case, we will generate a default elliptical geometry with 128 locations for our point transducers:

In [13]:
# # Create geometry
# num_locations = 64
# half_num_locations = num_locations#/2
# delta = space.limit[0]/(half_num_locations+1)
# problem.geometry.default('linear', num_locations,radius=[delta,delta],centre=[0.0e-3,0.0e-3])
# #problem.geometry.default('elliptical', num_locations)




In [14]:
num_locations = 64
problem.geometry.default('curvilinear', num = num_locations,start = [34.0e-3,0.0e-3],chord = 50e-3, pixel_size = 0.498e-3)

In [15]:
# # Create geometry
# num_locations = 128
# half_num_locations = num_locations
# problem.geometry.default('elliptical', num_locations)

Finally, we need to define what transducers in the geometry act as sources or as receivers. We do this by defining the `Acquisition`.

Because we have already saved our `Acquisition` as the output of the forward problem, we can simply reload it here instead of defining it again. The loaded information will contain the `Shot`s, the source excitations, and the observed data that is generated by the forward run.

In [16]:
# Create acquisitions
problem.acquisitions.load(path=problem.output_folder,project_name=problem.name, version=0)

In [17]:
#out = problem.acquisitions.load(path=problem.output_folder,project_name=problem.name, version=0)


Once complete, we can review our `Problem` by plotting its components:

In [18]:
# Plot
problem.plot()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

At this point, we have to define the image reconstruction process by creating an object of type `Optimisation`.

In [19]:
# # Create optimisation object
# optimisation = Optimisation()

In [20]:
# added new
pde = IsoAcousticDevito.remote(grid=problem.grid, len=runtime.num_workers,platform="nvidia-acc")

In [21]:
# added new
loss = L2DistanceLoss.remote(len=runtime.num_workers,platform="nvidia-acc")

In [22]:
# added new
step_size = 10
process_grad = ProcessGlobalGradient()
#process_model = ProcessModelIteration(min=1200., max=3000)#max=1700.)
process_model = ProcessModelIteration(min=300., max=2000)#max=1700.)
optimiser = GradientDescent(vp, step_size=step_size,
                            process_grad=process_grad,
                            process_model=process_model)

In [23]:
# new added
optimisation_loop = OptimisationLoop()

In [ ]:
# new added
# Specify a series of frequency bands, which we will introduce gradually 
# into the inversion in order to better condition it
f_min = 0.05e6
max_freqs = [0.5e6, 1.0e6, 1.5e6, 2.6e6]#[0.3e6, 0.4e6, 0.5e6, 0.6e6]#[0.03e6, 0.04e6, 0.05e6, 0.06e6]#

num_blocks = 4
num_iters = 16

# Start iterating over each block in the optimisation
for block, f_max in optimisation_loop.blocks(num_blocks, max_freqs):
    # We use pre-defined pipelines to process the source wavelets and the
    # observed and modelled traces. These will take care, among other things,
    # of filtering the data in frequency
    process_wavelets = ProcessWavelets.remote(f_min=f_min, f_max=f_max,
                                              len=runtime.num_workers)
    process_traces = ProcessTraces.remote(f_min=f_min, f_max=f_max,
                                          len=runtime.num_workers)

    # Proceed through every iteration in the block
    for iteration in block.iterations(num_iters):
        runtime.logger.info('Starting iteration %d (out of %d), '
                            'block %d (out of %d)' %
                            (iteration.id, block.num_iterations, block.id,
                             optimisation_loop.num_blocks))

        # Select some shots for this iteration
        shot_ids = problem.acquisitions.select_shot_ids(num=16, randomly=True)

        # Clear the gradient buffers of the variable
        vp.clear_grad()

        # Asynchronously loop over all the selected shot IDs
        @runtime.async_for(shot_ids)
        async def loop(worker, shot_id):
            runtime.logger.info('Giving shot %d to %s' % (shot_id, worker.uid))

            # Fetch one sub-problem corresponding to the shot ID
            sub_problem = problem.sub_problem(shot_id)
            wavelets = sub_problem.shot.wavelets
            observed = sub_problem.shot.observed

            # Pre-process the wavelets
            wavelets = process_wavelets(wavelets, runtime=worker)
            # Execute the PDE forward
            modelled = pde(wavelets, vp,attenuation_power = 2, problem=sub_problem, runtime=worker,platform="nvidia-acc")
            #modelled = pde(wavelets, vp,alpha,kernel = 'OT4',attenuation_power = 0,problem=sub_problem,runtime=worker)

            # Pre-process the modelled and observed traces
            traces = process_traces(modelled, observed, runtime=worker)
            
            # and use these pre-processed versions to calculate the
            # value of the loss function
            fun = await loss(traces.outputs[0], traces.outputs[1],
                             problem=sub_problem, runtime=worker,platform="nvidia-acc").result()

            iteration.add_fun(fun)
            runtime.logger.info('Functional value for shot %d: %s' % (shot_id, fun))

            # Now, we can calculate the gradient by executing the adjoint of the
            # forward process
            await fun.adjoint()

            runtime.logger.info('Retrieved gradient for shot %d' % sub_problem.shot_id)

        # Because this is an async loop, it needs to be awaited    
        _ = await loop
        # Update the vp with the calculated gradient by taking a step with the optimiser
        await optimiser.step()

        runtime.logger.info('Done iteration %d (out of %d), '
                            'block %d (out of %d) - Total loss %e' %
                            (iteration.id, block.num_iterations, block.id,
                             optimisation_loop.num_blocks, iteration.fun_value))
        runtime.logger.info('====================================================================')

# Plot the vp afterwards 
print("End of the simulation")
vp.plot()


HEAD            Starting iteration 0 (out of 16), block 0 (out of 4)
HEAD            Giving shot 36 to worker:0:0
HEAD            Giving shot 59 to worker:0:1
HEAD            Giving shot 6 to worker:0:2
HEAD            Giving shot 43 to worker:0:3
HEAD            Giving shot 9 to worker:0:4
WORKER:0:2      (ShotID 6) Preparing to run state for shot
WORKER:0:2      (ShotID 6) Estimated bandwidth for the propagated wavelet 0.075-0.685 MHz
WORKER:0:2      (ShotID 6) Spatial grid spacing (0.500 mm | 1 PPW) is higher than dispersion limit (0.110 mm | 5 PPW)
WORKER:0:2      (ShotID 6) Time grid spacing (0.100 μs | 37%) is below OT2 limit (0.121 μs)
WORKER:0:2      (ShotID 6) Selected undersampling level 3
WORKER:0:2      (ShotID 6) Selected time stepping scheme OT2
WORKER:0:2      Operator `acoustic_iso_state` configuration:
WORKER:0:2      	 * autotuning=['aggressive', 'runtime']
WORKER:0:2      	 * develop-mode=False
WORKER:0:2      	 * mpi=False
WORKER:0:2      	 * log-level=DEBUG
WORKER:

WORKER:0:0      	 * mpi=False
WORKER:0:0      	 * log-level=DEBUG
WORKER:0:0      	 * subs={h_x: 0.0005, h_y: 0.0005}
WORKER:0:0      	 * opt=advanced
WORKER:0:0      	 * platform=None
WORKER:0:0      	 * language=openmp
WORKER:0:0      	 * compiler=None
WORKER:0:0      Operator `acoustic_iso_state` generated in 2.93 s
WORKER:0:0        * lowering.Clusters: 1.31 s (44.8 %)
WORKER:0:0           * specializing.Clusters: 0.63 s (21.6 %)
WORKER:0:0        * lowering.Expressions: 0.85 s (29.1 %)
WORKER:0:0        * lowering.IET: 0.66 s (22.6 %)
WORKER:0:0      Flops reduction after symbolic optimization: [324 --> 222]
WORKER:0:0      Operator `acoustic_iso_state` fetched `/tmp/devito-jitcache-uid1004/2de14c15b65c709cddbb99bf89dd4c82da1b9bfb.c` in 0.22 s from jit-cache
WORKER:0:0      (ShotID 36) Running state equation for shot
WORKER:0:0      AutoTuner: could not perform any runs
WORKER:0:0      Operator `acoustic_iso_state` ran in 9.08 s
WORKER:0:0      Global performance: [OI=0.01, 5.87 G

WORKER:0:2      	 * autotuning=['aggressive', 'runtime']
WORKER:0:2      	 * develop-mode=False
WORKER:0:2      	 * mpi=False
WORKER:0:2      	 * log-level=DEBUG
WORKER:0:2      	 * subs={h_x: 0.0005, h_y: 0.0005}
WORKER:0:2      	 * opt=advanced
WORKER:0:2      	 * platform=None
WORKER:0:2      	 * language=openmp
WORKER:0:2      	 * compiler=None
WORKER:0:2      Operator `acoustic_iso_adjoint` generated in 2.13 s
WORKER:0:2        * lowering.Clusters: 0.90 s (42.4 %)
WORKER:0:2           * specializing.Clusters: 0.55 s (25.9 %)
WORKER:0:2        * lowering.Expressions: 0.60 s (28.3 %)
WORKER:0:2        * lowering.IET: 0.53 s (25.0 %)
WORKER:0:2      Flops reduction after symbolic optimization: [317 --> 218]
WORKER:0:2      Operator `acoustic_iso_adjoint` fetched `/tmp/devito-jitcache-uid1004/d35663002fa80140a5275b64e0865023582d71a7.c` in 0.23 s from jit-cache
WORKER:0:2      (ShotID 6) Running adjoint equation for shot
WORKER:0:2      AutoTuner: could not perform any runs
WORKER:0:2 

WORKER:0:1        * section1<<10000,1>,<10000,1>,<10000,1>,<10000,1>,<10000,1>> ran in 0.09 s [OI=3.32, 0.01 GFlops/s, 0.01 GPts/s]
WORKER:0:1        * section2<<10000,64>,<10000,64>,<10000,64>,<10000,64>,<10000,64>,<10000,64>> ran in 0.08 s [OI=3.84, 0.37 GFlops/s]
WORKER:0:1        * section3<10000,356,356> ran in 1.31 s [OI=0.75, 5.83 GFlops/s, 0.98 GPts/s]
WORKER:0:1      Performance[mode=advanced] arguments: {'nthreads': 4, 'nthreads_nonaffine': 4}
WORKER:0:1      (ShotID 35) Completed state equation run for shot
HEAD            Functional value for shot 35: loss 1.150912e+02 for shot 35
WORKER:0:2      (ShotID 8) Preparing to run state for shot
WORKER:0:2      (ShotID 8) Estimated bandwidth for the propagated wavelet 0.075-0.685 MHz
WORKER:0:2      (ShotID 8) Spatial grid spacing (0.500 mm | 1 PPW) is higher than dispersion limit (0.110 mm | 5 PPW)
WORKER:0:2      (ShotID 8) Time grid spacing (0.100 μs | 37%) is below OT2 limit (0.121 μs)
WORKER:0:2      (ShotID 8) Selected under

WORKER:0:4      AutoTuner: could not perform any runs
WORKER:0:4      Operator `acoustic_iso_adjoint` ran in 6.47 s
WORKER:0:4      Global performance: [OI=0.01, 7.46 GFlops/s, 0.20 GPts/s]
WORKER:0:4      Local performance:
WORKER:0:4        * section0<<10000,356,356>,<10000,0,356>,<10000,0,356>,<10000,356,0>,<10000,356,0>> ran in 4.28 s [OI=0.01, 10.67 GFlops/s, 0.30 GPts/s]
WORKER:0:4        * section1<<10000,64>,<10000,64>,<10000,64>,<10000,64>,<10000,64>> ran in 0.17 s [OI=3.32, 0.21 GFlops/s, 0.01 GPts/s]
WORKER:0:4        * section2<10000,356,356> ran in 2.02 s [OI=0.13, 1.26 GFlops/s]
WORKER:0:4      Performance[mode=advanced] arguments: {'nthreads': 4, 'nthreads_nonaffine': 4}
WORKER:0:4      (ShotID 4) Completed adjoint equation run for shot
HEAD            Retrieved gradient for shot 4
HEAD            Giving shot 12 to worker:0:4
WORKER:0:2      (ShotID 8) Preparing to run adjoint for shot
WORKER:0:2      (ShotID 8) Running adjoint equation for shot
WORKER:0:2      AutoTuner

...

WORKER:0:1        * section3<10000,356,356> ran in 0.84 s [OI=0.75, 9.14 GFlops/s, 1.53 GPts/s]
WORKER:0:1      Performance[mode=advanced] arguments: {'nthreads': 4, 'nthreads_nonaffine': 4}
WORKER:0:1      (ShotID 12) Completed state equation run for shot
HEAD            Functional value for shot 59: loss 1.207504e+02 for shot 59
HEAD            Functional value for shot 12: loss 1.237873e+02 for shot 12
WORKER:0:4      (ShotID 20) Preparing to run adjoint for shot
WORKER:0:4      (ShotID 20) Running adjoint equation for shot
WORKER:0:4      AutoTuner: could not perform any runs
WORKER:0:4      Operator `acoustic_iso_adjoint` ran in 4.52 s
WORKER:0:4      Global performance: [OI=0.01, 10.67 GFlops/s, 0.29 GPts/s]
WORKER:0:4      Local performance:
WORKER:0:4        * section0<<10000,356,356>,<10000,0,356>,<10000,0,356>,<10000,356,0>,<10000,356,0>> ran in 2.93 s [OI=0.01, 15.59 GFlops/s, 0.44 GPts/s]
WORKER:0:4        * section1<<10000,64>,<10000,64>,<10000,64>,<10000,64>,<10000,64>> r

WORKER:0:0      (ShotID 44) Completed state equation run for shot
WORKER:0:3      (ShotID 14) Preparing to run state for shot
WORKER:0:3      (ShotID 14) Estimated bandwidth for the propagated wavelet 0.592-1.785 MHz
WORKER:0:3      (ShotID 14) Spatial grid spacing (0.500 mm | 0 PPW) is higher than dispersion limit (0.041 mm | 5 PPW)
WORKER:0:3      (ShotID 14) Time grid spacing (0.100 μs | 40%) is below OT2 limit (0.113 μs)
WORKER:0:3      (ShotID 14) Selected undersampling level 2
WORKER:0:3      (ShotID 14) Selected time stepping scheme OT2
WORKER:0:3      (ShotID 14) Running state equation for shot
WORKER:0:3      AutoTuner: could not perform any runs
WORKER:0:3      Operator `acoustic_iso_state` ran in 4.94 s
WORKER:0:3      Global performance: [OI=0.01, 10.79 GFlops/s, 0.26 GPts/s]
WORKER:0:3      Local performance:
WORKER:0:3        * section0<<10000,356,356>,<10000,0,356>,<10000,0,356>,<10000,356,0>,<10000,356,0>> ran in 3.27 s [OI=0.01, 13.96 GFlops/s, 0.39 GPts/s]
WORKER:0:3 

WORKER:0:2        * section2<10000,356,356> ran in 1.37 s [OI=0.13, 1.87 GFlops/s]
WORKER:0:2      Performance[mode=advanced] arguments: {'nthreads': 4, 'nthreads_nonaffine': 4}
WORKER:0:2      (ShotID 52) Completed adjoint equation run for shot
HEAD            Retrieved gradient for shot 52
WORKER:0:1      (ShotID 17) Preparing to run adjoint for shot
WORKER:0:1      (ShotID 17) Running adjoint equation for shot
WORKER:0:1      AutoTuner: could not perform any runs
WORKER:0:1      Operator `acoustic_iso_adjoint` ran in 4.87 s
WORKER:0:1      Global performance: [OI=0.01, 9.91 GFlops/s, 0.27 GPts/s]
WORKER:0:1      Local performance:
WORKER:0:1        * section0<<10000,356,356>,<10000,0,356>,<10000,0,356>,<10000,356,0>,<10000,356,0>> ran in 3.17 s [OI=0.01, 14.42 GFlops/s, 0.41 GPts/s]
WORKER:0:1        * section1<<10000,64>,<10000,64>,<10000,64>,<10000,64>,<10000,64>> ran in 0.13 s [OI=3.32, 0.28 GFlops/s, 0.01 GPts/s]
WORKER:0:1        * section2<10000,356,356> ran in 1.58 s [OI=0.13

WORKER:0:3        * section1<<10000,1>,<10000,1>,<10000,1>,<10000,1>,<10000,1>> ran in 0.12 s [OI=3.32, 0.01 GFlops/s, 0.01 GPts/s]
WORKER:0:3        * section2<<10000,64>,<10000,64>,<10000,64>,<10000,64>,<10000,64>,<10000,64>> ran in 0.14 s [OI=3.84, 0.23 GFlops/s]
WORKER:0:3        * section3<10000,356,356> ran in 1.57 s [OI=0.75, 4.86 GFlops/s, 0.81 GPts/s]
WORKER:0:3      Performance[mode=advanced] arguments: {'nthreads': 4, 'nthreads_nonaffine': 4}
WORKER:0:3      (ShotID 48) Completed state equation run for shot
WORKER:0:2      (ShotID 39) Preparing to run state for shot
WORKER:0:2      (ShotID 39) Estimated bandwidth for the propagated wavelet 0.592-1.785 MHz
WORKER:0:2      (ShotID 39) Spatial grid spacing (0.500 mm | 0 PPW) is higher than dispersion limit (0.041 mm | 5 PPW)
WORKER:0:2      (ShotID 39) Time grid spacing (0.100 μs | 40%) is below OT2 limit (0.113 μs)
WORKER:0:2      (ShotID 39) Selected undersampling level 2
WORKER:0:2      (ShotID 39) Selected time stepping sch

WORKER:0:4      (ShotID 47) Completed adjoint equation run for shot
HEAD            Retrieved gradient for shot 39
HEAD            Giving shot 11 to worker:0:2
HEAD            Retrieved gradient for shot 47
HEAD            Giving shot 63 to worker:0:4


In [ ]:
# # new added
# for block, freq in optimisation_loop.blocks(num_blocks, max_freqs):
#     await adjoint(problem, pde, loss,
#                   optimisation_loop, optimiser, vp,
#                   num_iters=num_iters,
#                   select_shots=dict(num=16, randomly=True),
#                   f_min=0.05e6, f_max=freq)

# vp.plot()

In [ ]:
await mosaic.interactive('off')

We are interested in reconstructing the speed of sound of the medium, so we can create a corresponding optimisation variable that will be updated during the optimisation:

In [ ]:
# # Create optimisation variable
# optim_vp = Vp('vp', grid=problem.grid)
# optim_vp.extended_data[:] = vp.extended_data[:]

Lastly, we define an optimiser to update the variable.

In this case, we choose gradient descent with a constant step size:

In [ ]:
# # Create optimiser
# step_size = 100#10
# optimiser = GradientDescent(optim_vp, step=step_size)

# optimisation.add(optim_vp, optimiser)

Before we start executing our inversion, we need to start up the mosaic runtime:

In [ ]:
# await mosaic.interactive('on')

Now, we are ready to execute the problem.

To obtain a better posed optimisation, we use a multi-scale approach. We will start our inversion by using only low frequencies to construct our model. As the inversion progresses, we will add higher frequencies until we reach the maximum desired level of detail.

We can do this by dividing our optimisation in `Block`s and specifying a maximum frequency. Each `Block` will run for a number of specified iterations.

During each iteration, only a subset of the shots will be used by defining the `select_shots` field. In this case, 16 shots will be used for each iteration, and they will be selected randomly without replacement.

In [ ]:
# # # Run optimisation
# # max_freqs = [0.6e6]

# # for freq, block in zip(max_freqs, optimisation.blocks(1)):
# #     block.config(num_iterations=8,
# #                  f_min=0.05e6, f_max=freq,
# #                  min=1400., max=1700.,
# #                  select_shots={'num': 16, 'randomly': True})

# #     await optimisation.run(block, problem)


# # Run optimisation
# max_freqs = [0.8e6,1.2e6,1.6e6,2.1e6]#[0.3e6, 0.4e6, 0.5e6, 0.6e6]#[0.8e6,1.2e6,1.6e6,2.1e6]

# for freq, block in zip(max_freqs, optimisation.blocks(4)):
#     block.config(num_iterations=8,
#                  f_min=0.05e6, f_max=freq, # instead of f_min = 0.05e6
#                  min=1400, max=5500,
#                  select_shots={'num': 16, 'randomly': True})

#     await optimisation.run(block, problem)

Before closing, we should tear down the mosaic runtime.

In [ ]:
# await mosaic.interactive('off')

In [ ]:
problem.plot()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure()
problem.medium.plot()

In [ ]:
plt.savefig('./FinalSim/VP/Abdomen_A2_30_120_150')

In [ ]:
import numpy as np
x = problem.medium.vp.data
x2 = np.array(x)

print(np.max(x2))
plt.figure()
plt.imshow(x2,cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
import scipy.misc
from scipy import ndimage
import matplotlib.pyplot as plt

img = x2 
# img = scipy.misc.face()  # lena is not included in scipy 0.19.1
plt.figure()

degree= 90
rotated_img = ndimage.rotate(img, degree)
plt.imshow(rotated_img, cmap=plt.cm.gray)
plt.axis('off')
plt.show()
#plt.savefig('./output/EXP/medical/Liver/112.pdf')
np.save('./FinalSim/Sim_SoS/Abdomen_A2_30_120_150.npy',rotated_img )
import PIL
from PIL import Image
img = rotated_img
mn = np.min(img)
mx = np.max(img)
img = ((img-np.ones(img.shape)*mn)/(mx-mn))*255
img = Image.fromarray(img)
img = img.convert("L")
print(type(img))
#img.save('./output/EXP/medical/112.jpg')
img.save('./FinalSim/Sim_SoS/Abdomen_A2_30_120_150.png')